In [18]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, Sequence, Annotated, Literal
from langgraph.graph import add_messages
from langgraph.types import Command
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import BaseMessage,HumanMessage,AIMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv  

In [3]:
load_dotenv()

True

In [16]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

In [11]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]
    email : Annotated[str, "Email address mentioned in the content"]
    role : Annotated[str, "Role mentioned in the content"]
    job_description : Annotated[str, "Job description mentioned in the content"]

EXTRACTOR AGENT'S CODE

In [19]:
extractor_prompt = PromptTemplate(
    input_variables=["content"],
    template="""You are an expert in extracting information from text.
    Extract the email address, role, and job description from the following content:

    {content}

    Provide the extracted information in the following format:
    Email: <email>
    Role: <role>
    Job Description: <job_description>""",
)

extractor_chain = extractor_prompt | llm

def extractor_agent(state: State) -> Command[Literal["rag_agent"]]:
    "An agent that extracts email, role and job description from the content provided by the user in order to send to rag agent which extracts relevant information from the resume"
    messages = state.get("messages", [])
    if not messages: #edge case to handle empty messages
        print("No messages found in state.")
        return "error_or_exit"
    last_message = messages[-1].content
    response = extractor_chain.invoke({"content": last_message})
    return Command(
        "rag_agent",
        update={
            "email": response.get("Email", ""),
            "role": response.get("Role", ""),
            "job_description": response.get("Job Description", ""),
            "messages": messages,
        },
    )

RAG AGENT CODE

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool